In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import pandas as pd
import requests
import json
import time
from datetime import datetime
from urllib.parse import quote
import os


In [ ]:

def get_coordinates_from_roadaddress(address, api_key):
    """
    V-World API를 사용하여 도로명주소로부터 좌표를 얻는 함수
    
    Args:
        address (str): 도로명주소
        api_key (str): V-World API 인증키
    
    Returns:
        tuple: (경도, 위도, 상태메시지)
    """
    apiurl = "https://api.vworld.kr/req/address?"
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "refine" : "true", # refine 파라미터는 false로 설정하여 정확한 주소를 사용
        "key": api_key
    }
    
    try:
        response = requests.get(apiurl, params=params, timeout=10)
        if response.status_code == 200:
            data = response.json()
            
            # API 응답 구조 확인
            if 'response' in data:
                if 'status' in data['response'] and data['response']['status'] == 'OK':
                    if 'result' in data['response'] and 'point' in data['response']['result']:
                        point = data['response']['result']['point']
                        x = float(point.get('x', 0))  # 경도
                        y = float(point.get('y', 0))  # 위도
                        return x, y, "Success"
                    else:
                        return None, None, "No coordinates found"
                else:
                    error_msg = data['response'].get('status', 'Unknown error')
                    return None, None, f"API Error: {error_msg}"
            else:
                return None, None, f"Unexpected response format: {data}"
        else:
            return None, None, f"HTTP Error: {response.status_code}"
    
    except requests.exceptions.Timeout:
        return None, None, "Request timeout"
    except requests.exceptions.RequestException as e:
        return None, None, f"Request error: {str(e)}"
    except Exception as e:
        return None, None, f"Exception: {str(e)}"


In [ ]:

def get_coordinates_from_pointaddress(address, api_key):
    """
    V-World API를 사용하여 도로명주소로부터 좌표를 얻는 함수
    
    Args:
        address (str): 도로명주소
        api_key (str): V-World API 인증키
    
    Returns:
        tuple: (경도, 위도, 상태메시지)
    """
    apiurl = "https://api.vworld.kr/req/address?"
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "parcel",  # 지번주소를 위한 type 파라미터
        "refine" : "true", # refine 파라미터는 false로 설정하여 정확한 주소를 사용
        "key": api_key
    }
    
    try:
        response = requests.get(apiurl, params=params, timeout=10)
        if response.status_code == 200:
            data = response.json()
            
            # API 응답 구조 확인
            if 'response' in data:
                if 'status' in data['response'] and data['response']['status'] == 'OK':
                    if 'result' in data['response'] and 'point' in data['response']['result']:
                        point = data['response']['result']['point']
                        x = float(point.get('x', 0))  # 경도
                        y = float(point.get('y', 0))  # 위도
                        return x, y, "Success"
                    else:
                        return None, None, "No coordinates found"
                else:
                    error_msg = data['response'].get('status', 'Unknown error')
                    return None, None, f"API Error: {error_msg}"
            else:
                return None, None, f"Unexpected response format: {data}"
        else:
            return None, None, f"HTTP Error: {response.status_code}"
    
    except requests.exceptions.Timeout:
        return None, None, "Request timeout"
    except requests.exceptions.RequestException as e:
        return None, None, f"Request error: {str(e)}"
    except Exception as e:
        return None, None, f"Exception: {str(e)}"


In [ ]:

def main():
    """
    메인 함수: 엑셀 파일에서 도로명주소를 읽고 좌표를 변환
    """
    # 설정
    excel_file = r"A.기초정보\도로명주소_좌표변환_결과_오류사업장_1차.xlsx"  # 엑셀 파일 경로
    sheet_name = "주소변환"  # 시트명 (첫 번째 시트)
    api_key = "D7EE0C72-613F-31B0-A251-C5DCE08FE29C"  # ※※※ 실제 V-World API 키로 교체 필요 ※※※
    
    # API 키 확인
    if api_key == "YOUR_API_KEY_HERE":
        print("⚠️  경고: V-World API 키를 설정해주세요!")
        print("api_key 변수에 실제 발급받은 API 키를 입력하세요.")
        print("API 키 발급: https://www.vworld.kr (회원가입 > 오픈API > 인증키 발급)")
        return
    
    try:
        # 엑셀 파일 읽기
        print("📖 엑셀 파일 읽는 중...")
        df = pd.read_excel(excel_file, sheet_name=sheet_name)
        print(f"✅ 파일 읽기 완료! (총 {len(df)}행)")
        print(f"📋 컬럼: {list(df.columns)}")
        
        # 도로명주소 컬럼 확인
        if '도로명주소' not in df.columns:
            print("❌ 오류: '도로명주소' 컬럼을 찾을 수 없습니다!")
            print("사용 가능한 컬럼:", list(df.columns))
            return
        
        # 빈 주소 제거
        valid_roadaddresses = df[df['도로명주소'].notna() & (df['도로명주소'] != "")]
        valid_pointaddresses = df[df['지번주소'].notna() & (df['지번주소'] != "")]
        print(f"📍 유효한 주소: {len(valid_roadaddresses)}개")
        print(f"📍 유효한 주소: {len(valid_pointaddresses)}개")
        
        # 결과 저장 리스트
        results = []
        
        print("\n🌍 좌표 변환 시작...")
        print("-" * 80)
        
        # 각 주소에 대해 좌표 조회
        for idx, row in valid_roadaddresses.iterrows():
            roadaddress = row['도로명주소'].strip()
            pointaddress = row['지번주소'].strip()
            business_name = row.get('사업장명', 'Unknown')
            
            print(f"[{idx+1:3d}] {business_name[:20]:<20} | {roadaddress}")
            
            # 좌표 조회
            x, y, status = get_coordinates_from_roadaddress(roadaddress, api_key)

            if status != "Success":
                x, y, status = get_coordinates_from_pointaddress(pointaddress, api_key)
            
            results.append({
                '행번호': idx + 1,
                '시도': row.get('시도', ''),
                '지역': row.get('지역', ''),
                '사업장명': business_name,
                '사업장코드': row.get('사업장코드', row.get('사업장코드', '')),
                '우편번호': row.get('우편번호', ''),
                '지번주소': pointaddress,
                '도로명주소': roadaddress,
                '경도(X)': x,
                '위도(Y)': y,
                '변환상태': status
            })
            
            if status == "Success":
                print(f"      ✅ 경도: {x:.6f}, 위도: {y:.6f}")
            else:
                print(f"      ❌ {status}")
            
            # API 호출 제한을 위한 대기 (초당 1회 정도)
            time.sleep(1.0)
        
        # 결과를 DataFrame으로 변환
        result_df = pd.DataFrame(results)
        
        # 결과 파일 저장
        
        output_file = f"도로명주소_좌표변환_결과_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        result_df.to_excel(output_file, index=False, engine='openpyxl')
        print(f"\n💾 결과 저장 완료: {output_file}")
        
        # 통계 출력
        print("\n📊 변환 결과 요약")
        print("=" * 50)
        successful = result_df[result_df['변환상태'] == 'Success'].shape[0]
        total = len(result_df)
        success_rate = (successful / total * 100) if total > 0 else 0
        
        print(f"총 주소 개수: {total:,}개")
        print(f"성공적 변환: {successful:,}개")
        print(f"실패한 변환: {total - successful:,}개")
        print(f"성공률: {success_rate:.1f}%")
        
        # 실패 사례 분석
        failed_df = result_df[result_df['변환상태'] != 'Success']
        if len(failed_df) > 0:
            print(f"\n❌ 실패 사례 ({len(failed_df)}개):")
            failure_reasons = failed_df['변환상태'].value_counts()
            for reason, count in failure_reasons.items():
                print(f"  - {reason}: {count}개")
            
            print("\n실패한 주소 예시:")
            for idx, row in failed_df.head(5).iterrows():
                print(f"  • {row['도로명주소']} ({row['변환상태']})")
        
        # 성공 사례 샘플
        success_df = result_df[result_df['변환상태'] == 'Success']
        if len(success_df) > 0:
            print(f"\n✅ 성공 사례 샘플:")
            for idx, row in success_df.head(3).iterrows():
                print(f"  • {row['도로명주소']}")
                print(f"    → 경도: {row['경도(X)']:.6f}, 위도: {row['위도(Y)']:.6f}")
        
        return result_df
        
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {excel_file}")
        print("현재 디렉토리에 파일이 있는지 확인해주세요.")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return None

def test_api_key(api_key):
    """
    API 키가 유효한지 테스트하는 함수
    """
    test_address = "서울특별시 종로구 세종대로 209"
    print(f"🔑 API 키 테스트 중... (테스트 주소: {test_address})")
    
    x, y, status = get_coordinates_from_roadaddress(test_address, api_key)
    
    if status == "Success":
        print(f"✅ API 키가 유효합니다! 좌표: ({x}, {y})")
        return True
    else:
        print(f"❌ API 키 테스트 실패: {status}")
        return False

if __name__ == "__main__":
    print("🌏 V-World API를 사용한 도로명주소 좌표 변환 프로그램")
    print("=" * 60)
    
    # 필요한 라이브러리 설치 안내
    try:
        import pandas as pd
        import requests
        import openpyxl


    except ImportError as e:
        print("❌ 필요한 라이브러리가 설치되지 않았습니다:")
        print("다음 명령어로 설치해주세요:")
        print("pip install pandas requests openpyxl")
        exit(1)
    
    # 메인 실행
    main()

# ====================================================================
# 사용 방법:
# ====================================================================
# 1. V-World 사이트(https://www.vworld.kr)에서 회원가입
# 2. 오픈API > 인증키 > 인증키 발급에서 API 키 발급
# 3. 위 코드의 api_key 변수에 발급받은 키 입력
# 4. 엑셀 파일과 같은 폴더에 이 스크립트 저장 후 실행
# 
# 결과 파일: "도로명주소_좌표변환_결과.xlsx"
# ====================================================================